In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
Data =  pd.read_json('/content/drive/My Drive/Sarcasm Detection/Arabic News Headlines Dataset.json', orient = 'split')
Data.head()

,headlines,topic,length,is_sarcastic
0,طالب يصبح متعلّماً بعد دراسته وحدة عن القائد ف...,Society,12,1
1,سموحة يقلص الفارق ويسجل هدفا ثانيا في شباك الم...,Sports,11,0
2,ليتها الحدود – الولايات المتحدة تهدد بفرض عقوب...,Misc,17,1
3,وزيرة الهجرة تتواصل مع سفير مصر في نيوزيلندا ل...,None,14,0
4,ليتها الحدود – مصر تقرر إرسال أعضاء السائح الب...,Misc,12,1


In [0]:
X = Data['headlines']
Labels = Data['is_sarcastic']

In [0]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = ['acc'])

model.fit(x_train, y_train, 
          validation_split = 0.2, 
          batch_size = 32, 
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', accuracy)

Instructions for updating:
Use tf.cast instead.
Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 18s 5ms/step - loss: 0.5461 - acc: 0.7368 - val_loss: 0.3778 - val_acc: 0.8362
Epoch 2/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3371 - acc: 0.8475 - val_loss: 0.3241 - val_acc: 0.8561
Epoch 3/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3036 - acc: 0.8670 - val_loss: 0.3050 - val_acc: 0.8651
Epoch 4/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2844 - acc: 0.8770 - val_loss: 0.2950 - val_acc: 0.8681
Epoch 5/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2740 - acc: 0.8823 - val_loss: 0.2901 - val_acc: 0.8731
Epoch 6/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2656 - acc: 0.8870 - val_loss: 0.2835 - val_acc: 0.8721
Epoch 7/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2581 - acc: 0.8

In [0]:
import numpy as np
max_sequence_length = np.max(Data['length'])
print("max length is:", max_sequence_length)

max length is: 34


In [0]:
import keras
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=30000)
tokenizer.fit_on_texts(X)
sequences = tokenizer.texts_to_sequences(X)

word_index = tokenizer.word_index   # a dictionary of each word and its index
print('Found %s unique tokens.' % len(word_index))

Using TensorFlow backend.


Found 20595 unique tokens.


In [0]:
from keras.preprocessing.sequence import pad_sequences
data = pad_sequences(sequences, maxlen=max_sequence_length)
print('Shape of data tensor:', data.shape)

Shape of data tensor: (6669, 34)


In [0]:
from keras.utils import to_categorical
labels = to_categorical(np.asarray(Labels))  ## one hot of the output
print('Shape of label tensor:', labels.shape)

Shape of label tensor: (6669, 2)


In [0]:
import pickle as p
fastText = p.load(open('/content/drive/My Drive/Sarcasm Detection/FastText_Wiki_Embeddings.p', 'rb'))

In [0]:
# prepare embedding matrix  (matrix of embeddings of the words in our dataset)
num_words = len(word_index) + 1
# initialize the np array
unKnown = list()
embedding_matrix = np.zeros((num_words, 300))
l=0
# find the embeddings from the pre trained embedding matrix of fast text wiki
for word, i in word_index.items():
    embedding_vector = fastText.get(word)
    if embedding_vector is not None:
        l+=1
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector
    else:
      unKnown.append(word)
print("the number of words found in fast text embeddings =", l)
print("the number of words not found in fast text embeddings =", 20595- l)

the number of words found in fast text embeddings = 17289
the number of words not found in fast text embeddings = 3306


In [0]:
from keras.initializers import Constant
from keras.layers import Input, Embedding, Conv1D, GlobalMaxPooling2D, Dense, Dropout, LSTM, Bidirectional
from keras.models import Model

In [0]:
# shuffle the data

indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]


## split the data into training, testing

nb_validation_samples = int(0.25* data.shape[0])

x_train = data[:-nb_validation_samples]
y_train = labels[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = labels[-nb_validation_samples:]

In [0]:
embedding_layer = Embedding(num_words,
                            300,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=max_sequence_length,
                            trainable=False)

In [0]:
# the model

sequence_input = Input(shape=(max_sequence_length,), dtype='int32')             ## input layer
embedded_sequences = embedding_layer(sequence_input)                            ## embeddings layer
X = Conv1D(300, 3, activation='relu')(embedded_sequences)                       ## CNN layer
X = Conv1D(300, 3, activation='relu')(X)                                        ## CNN layer
X = Bidirectional(LSTM(200, return_sequences=False), merge_mode='concat')(X)    ## BiLSTM layer
X = Dropout(0.5)(X)                                                             ## Dropout layer
X = Dense(300, activation = "sigmoid")(X)                                       ## Fully Connected layer                     
preds = Dense(2, activation='softmax')(X)                                       ## softmax layer

model = Model(sequence_input, preds)



In [0]:
from keras import backend as K

def recall(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall


      
def precision(y_true, y_pred):
        
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
      

def f1(y_true, y_pred):
    
    Precision = precision(y_true, y_pred)
    Recall = recall(y_true, y_pred)
    f1 = 2*((Precision*Recall)/(Precision+Recall+K.epsilon()))
    
    return f1

In [0]:
adam = keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.3, amsgrad=False)

In [0]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [precision])
model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, prec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', prec)


Instructions for updating:
Use tf.cast instead.
Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 20s 5ms/step - loss: 0.5348 - precision: 0.7308 - val_loss: 0.3875 - val_precision: 0.8142
Epoch 2/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.3272 - precision: 0.8530 - val_loss: 0.3256 - val_precision: 0.8501
Epoch 3/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2883 - precision: 0.8753 - val_loss: 0.3222 - val_precision: 0.8561
Epoch 4/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2686 - precision: 0.8848 - val_loss: 0.2914 - val_precision: 0.8771
Epoch 5/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2581 - precision: 0.8893 - val_loss: 0.2904 - val_precision: 0.8761
Epoch 6/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2476 - precision: 0.8945 - val_loss: 0.2812 - val_precision: 0.8821
Epoch 7/50
4001/4001 [

In [0]:
model.compile(loss = 'categorical_crossentropy',
              optimizer = adam,
              metrics = [recall])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, rec = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test accuracy is: ', rec)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.4967 - recall: 0.7461 - val_loss: 0.3500 - val_recall: 0.8482
Epoch 2/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.3190 - recall: 0.8528 - val_loss: 0.3201 - val_recall: 0.8531
Epoch 3/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2880 - recall: 0.8695 - val_loss: 0.3202 - val_recall: 0.8501
Epoch 4/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2724 - recall: 0.8783 - val_loss: 0.3019 - val_recall: 0.8621
Epoch 5/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2618 - recall: 0.8865 - val_loss: 0.2918 - val_recall: 0.8721
Epoch 6/50
4001/4001 [==============================] - 13s 3ms/step - loss: 0.2546 - recall: 0.8873 - val_loss: 0.2819 - val_recall: 0.8701
Epoch 7/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.2481 - recall: 0.8915 - val

In [30]:
model.compile(loss = 'categorical_crossentropy',
          optimizer = adam, 
          metrics = [f1])

model.fit(x_train, y_train,
          validation_split = 0.2,
          batch_size = 32,
          epochs = 50)

loss, accuracy = model.evaluate(x_test, y_test)
print('test loss is:', loss)
print('test f1 score is: ', accuracy)

Train on 4001 samples, validate on 1001 samples
Epoch 1/50
4001/4001 [==============================] - 16s 4ms/step - loss: 0.4953 - f1: 0.7498 - val_loss: 0.3465 - val_f1: 0.8412
Epoch 2/50
4001/4001 [==============================] - 15s 4ms/step - loss: 0.3098 - f1: 0.8648 - val_loss: 0.3091 - val_f1: 0.8661
Epoch 3/50
4001/4001 [==============================] - 14s 4ms/step - loss: 0.2806 - f1: 0.8788 - val_loss: 0.3006 - val_f1: 0.8641
Epoch 4/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2668 - f1: 0.8880 - val_loss: 0.2866 - val_f1: 0.8821
Epoch 5/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2599 - f1: 0.8898 - val_loss: 0.2844 - val_f1: 0.8801
Epoch 6/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2512 - f1: 0.8945 - val_loss: 0.2799 - val_f1: 0.8801
Epoch 7/50
4001/4001 [==============================] - 14s 3ms/step - loss: 0.2448 - f1: 0.8963 - val_loss: 0.2779 - val_f1: 0.8771
Epoch 8/50
4001/4001 

In [0]:
 model.save_weights('/content/drive/My Drive/Sarcasm Detection/Plots/CNN_BiLSTM_optimized_Weights.h5')